### References

[1] R. Alsod, "<i>Cosine Similarity between 2 Number Lists,</i>" Stack Overflow, 2013. Link: https://stackoverflow.com/questions/18424228/cosine-similarity-between-2-number-lists [Last Accessed 17 Feb 2022]

[2] Justin1904, “<i>Justin1904/CMU-MultimodalSDK-Tutorials,</i>” GitHub, 2019. Link: https://github.com/Justin1904/CMU-MultimodalSDK-Tutorials/blob/master/tutorial_interactive.ipynb [Last Accessed 15 Feb 2022]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from operator import index
import sys 
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os
import tarfile
import urllib
import pandas as pd
import urllib.request

from shutil import copyfile
import glob
import pickle

import h5py
import cv2
from scipy.io import wavfile
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
! ls

drive  sample_data


In [ ]:
%cd /content/drive/MyDrive/Dissertation

/content/drive/MyDrive/Dissertation


In [ ]:
# !git clone https://github.com/A2Zadeh/CMU-MultimodalSDK.git

In [ ]:
%cd CMU-MultimodalSDK

/content/drive/MyDrive/Dissertation/CMU-MultimodalSDK


In [ ]:
import sys, os
sys.path.append(os.getcwd())

In [ ]:
!pip install h5py validators tqdm numpy argparse requests colorama

In [ ]:
import mmsdk
from mmsdk import mmdatasdk as md
import math

In [ ]:
dataMosei_train_text = h5py.File("/content/drive/MyDrive/Dissertation/dataset/data/text_train_emb.h5", "r")

In [ ]:
text_trainKeys = []

for key in dataMosei_train_text.keys():
    text_trainKeys.append(key)

print(text_trainKeys)

['d1']


In [ ]:
len

<function len>

In [ ]:
print("-----------Create train array-----------")

textTrainArr = np.array(dataMosei_train_text.get("d1"))
print(textTrainArr.shape)
print(textTrainArr[45].shape)
print(textTrainArr[67][17].shape)

-----------Create train array-----------
(15290, 20, 300)
(20, 300)
(300,)


In [ ]:
print("-----------Importing raw words + their glove embeddings from SDK-----------")

raw_words = 'CMU_MOSEI_TimestampedWords'
raw_words_embedded = 'CMU_MOSEI_TimestampedWordVectors'

raw_features = [raw_words]
raw_features_glove = [raw_words_embedded]

word_dict = {word: os.path.join('/content/drive/MyDrive/Dissertation/dataset/data/', word) + '.csd' for word in raw_features}          #[2]
word_dataset = md.mmdataset(word_dict)

word_dict_glove = {emb: os.path.join('/content/drive/MyDrive/Dissertation/dataset/data/', emb) + '.csd' for emb in raw_features_glove}    #[2]
glove_dataset = md.mmdataset(word_dict_glove)

-----------Importing raw words + their glove embeddings from SDK-----------
[2022-04-13 03:51:50.736] | Success | Computational sequence read from file /content/drive/MyDrive/Dissertation/dataset/data/CMU_MOSEI_TimestampedWords.csd ...
[2022-04-13 03:51:52.101] | Status  | Checking the integrity of the <words> computational sequence ...
[2022-04-13 03:51:52.102] | Status  | Checking the format of the data in <words> computational sequence ...


[2022-04-13 03:51:57.897] | Success | <words> computational sequence data in correct format.
[2022-04-13 03:51:57.897] | Status  | Checking the format of the metadata in <words> computational sequence ...
[2022-04-13 03:51:57.898] | Warning | <words> computational sequence does not have all the required metadata ... continuing 
[2022-04-13 03:51:57.898] | Success | Dataset initialized successfully ... 
[2022-04-13 03:51:58.380] | Success | Computational sequence read from file /content/drive/MyDrive/Dissertation/dataset/data/CMU_MOSEI_TimestampedWordVectors.csd ...
[2022-04-13 03:52:14.530] | Status  | Checking the integrity of the <glove_vectors> computational sequence ...
[2022-04-13 03:52:14.530] | Status  | Checking the format of the data in <glove_vectors> computational sequence ...


[2022-04-13 03:52:18.949] | Success | <glove_vectors> computational sequence data in correct format.
[2022-04-13 03:52:18.949] | Status  | Checking the format of the metadata in <glove_vectors> computational sequence ...
[2022-04-13 03:52:18.950] | Warning | <glove_vectors> computational sequence does not have all the required metadata ... continuing 
[2022-04-13 03:52:18.950] | Success | Dataset initialized successfully ... 


In [ ]:
print("RAW WORDS PRINTS")

print(list(word_dataset.keys()))
print("=" * 80)
print(list(word_dataset[raw_words].keys())[:10])
print("=" * 80)

print(len(list(word_dataset[raw_words].keys())))
print("=" * 80)

some_id = list(word_dataset[raw_words].keys())[0]         ########################################
print(some_id)
print("=" * 80)

print(list(word_dataset[raw_words][some_id].keys()))
print("=" * 80)
print(list(word_dataset[raw_words][some_id]['intervals'].shape))
print("=" * 80)
print(list(word_dataset[raw_words][some_id]['features'].shape))
print("=" * 80)
print(list(word_dataset[raw_words][some_id]['features'][4]))
print("=" * 80)
print(list(word_dataset[raw_words][some_id]['intervals'][4]))
print("=" * 80)

RAW WORDS PRINTS
['CMU_MOSEI_TimestampedWords']
['--qXJuDtHPw', '-3g5yACwYnA', '-3nNcZdcdvU', '-571d8cVauQ', '-6rXp3zJ3kc', '-9YyBTjo1zo', '-9y-fZ3swSY', '-AUZQgSxyPQ', '-Alixo7euuU', '-Eqdz5y4pEY']
3837
--qXJuDtHPw
['features', 'intervals']
[183, 2]
[183, 1]
[b'there']
[0.591156462585, 0.780725623583]


In [ ]:
print("GLOVE EMBEDDINGS PRINTS")

print(list(glove_dataset.keys()))
print("=" * 80)
print(list(glove_dataset[raw_words_embedded].keys())[:10])
print("=" * 80)

print(len(list(glove_dataset[raw_words_embedded].keys())))
print("=" * 80)

some_id = list(glove_dataset[raw_words_embedded].keys())[0]         ########################################
print(some_id)
print("=" * 80)

print(list(glove_dataset[raw_words_embedded][some_id].keys()))
print("=" * 80)
print(list(glove_dataset[raw_words_embedded][some_id]['intervals'].shape))
print("=" * 80)
print(list(glove_dataset[raw_words_embedded][some_id]['features'].shape))
print("=" * 80)

GLOVE EMBEDDINGS PRINTS
['CMU_MOSEI_TimestampedWordVectors']
['--qXJuDtHPw', '-3g5yACwYnA', '-3nNcZdcdvU', '-571d8cVauQ', '-6rXp3zJ3kc', '-9YyBTjo1zo', '-9y-fZ3swSY', '-AUZQgSxyPQ', '-Alixo7euuU', '-Eqdz5y4pEY']
3837
--qXJuDtHPw
['features', 'intervals']
[183, 2]
[183, 300]


In [ ]:
print("-----------Creating the cosine similarity function-----------")

#[1]
def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if(math.sqrt(sumxx*sumyy) != 0.0):
        return sumxy/math.sqrt(sumxx*sumyy)

print("FUNCTION CREATED")

-----------Creating the cosine similarity function-----------
FUNCTION CREATED


In [ ]:
len(textTrainArr)

15290

In [ ]:
embWordDict = {}

In [ ]:
for k in range(len(list(word_dataset[raw_words].keys()))):
   vid_id = list(word_dataset[raw_words].keys())[k]
   for j in range(len( ((glove_dataset[raw_words_embedded][vid_id]['features'])) )):
     embWordDict[tuple(((glove_dataset[raw_words_embedded][vid_id]['features'][j])))] = (word_dataset[raw_words][vid_id]['features'][j])


print("Process Completed")

 
DONE


In [ ]:
print(len(embWordDict))
print(list(embWordDict.items())[:4])

20549
[((0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

https://stackoverflow.com/questions/30629429/json-dump-gives-me-typeerror-keys-must-be-a-string 

In [ ]:
with open("/content/drive/MyDrive/Dissertation/dataset/data/rawWithEmbeddings.pkl", "wb") as fp:
  pickle.dump(embWordDict, fp)

In [ ]:
with open("/content/drive/MyDrive/Dissertation/dataset/data/rawWithEmbeddings.pkl", "rb") as fp:
  rawWordEmbDict = pickle.load(fp)

print(type(rawWordEmbDict))
print(len(rawWordEmbDict))
print(list(rawWordEmbDict.items())[:4])

<class 'dict'>
20549
[((0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [ ]:
import csv

In [ ]:
columnName = ['Sentence'] 

https://stackoverflow.com/questions/47307048/remove-commas-in-list-and-make-a-sentence-with-list-items

In [ ]:
arr1 = []
for sentence in range(7645):
  print("SENTENCE NUMBER - {}".format(sentence))
  arr_words1 = []
  arr_words_no_comma1 = []
  for word in range(20):
    for j in range(len(list(rawWordEmbDict.keys()))):
      flag = False
      if(tuple(textTrainArr[sentence][word])) in rawWordEmbDict:
        flag = True
        arr_words1.append(rawWordEmbDict[tuple(textTrainArr[sentence][word])][0].decode('UTF-8'))
      if(flag == True):
        break
  sent = ' '.join(arr_words1)
  arr_words_no_comma1.append(sent)
  arr1.append(arr_words_no_comma1)

print("Done")

Streaming output truncated to the last 5000 lines.
SENTENCE NUMBER - 2646
SENTENCE NUMBER - 2647
SENTENCE NUMBER - 2648
SENTENCE NUMBER - 2649
SENTENCE NUMBER - 2650
SENTENCE NUMBER - 2651
SENTENCE NUMBER - 2652
SENTENCE NUMBER - 2653
SENTENCE NUMBER - 2654
SENTENCE NUMBER - 2655
SENTENCE NUMBER - 2656
SENTENCE NUMBER - 2657
SENTENCE NUMBER - 2658
SENTENCE NUMBER - 2659
SENTENCE NUMBER - 2660
SENTENCE NUMBER - 2661
SENTENCE NUMBER - 2662
SENTENCE NUMBER - 2663
SENTENCE NUMBER - 2664
SENTENCE NUMBER - 2665
SENTENCE NUMBER - 2666
SENTENCE NUMBER - 2667
SENTENCE NUMBER - 2668
SENTENCE NUMBER - 2669
SENTENCE NUMBER - 2670
SENTENCE NUMBER - 2671
SENTENCE NUMBER - 2672
SENTENCE NUMBER - 2673
SENTENCE NUMBER - 2674
SENTENCE NUMBER - 2675
SENTENCE NUMBER - 2676
SENTENCE NUMBER - 2677
SENTENCE NUMBER - 2678
SENTENCE NUMBER - 2679
SENTENCE NUMBER - 2680
SENTENCE NUMBER - 2681
SENTENCE NUMBER - 2682
SENTENCE NUMBER - 2683
SENTENCE NUMBER - 2684
SENTENCE NUMBER - 2685
SENTENCE NUMBER - 2686
SENTEN

https://www.geeksforgeeks.org/writing-csv-files-in-python/

In [ ]:
with open("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Train.csv", 'w') as csvfp: 
    csvwriter = csv.writer(csvfp)  
    csvwriter.writerow(columnName)
    csvwriter.writerows(arr1)

In [ ]:
dfTrainPart1 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Train.csv")
display(dfTrainPart1.head())

,Sentence
0,independence we are only bound by human rights...
1,sp sp sp sp we scrutinize what governments wha...
2,sp sp sp sp sp sp sp sp sp sp sp sp sp sp inde...
3,sp sp sp sp work we help the states to impleme...
4,us to do that effectively we need to be indepe...


In [ ]:
dfTrainPart1.drop("totalwords", axis=1, inplace=True)
dfTrainPart1.drop("Sentence Length", axis=1, inplace=True)

https://stackoverflow.com/questions/49984905/count-number-of-words-per-row

In [ ]:
dfTrainPart1['Sentence Length'] = [len(x.split()) for x in dfTrainPart1['Sentence'].tolist()]
display(dfTrainPart1.head())
dfTrainPart1.to_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Train.csv", index=False)

,Sentence,Sentence Length
0,independence we are only bound by human rights...,19
1,sp sp sp sp we scrutinize what governments wha...,20
2,sp sp sp sp sp sp sp sp sp sp sp sp sp sp inde...,20
3,sp sp sp sp work we help the states to impleme...,20
4,us to do that effectively we need to be indepe...,20


In [ ]:
arr2 = []
for sentence in range(7645, 15290):
  print("SENTENCE NUMBER - {}".format(sentence))
  arr_words2 = []
  arr_words_no_comma2 = []
  for word in range(20):
    for j in range(len(list(rawWordEmbDict.keys()))):
      flag = False
      if(tuple(textTrainArr[sentence][word])) in rawWordEmbDict:
        flag = True
        arr_words2.append(rawWordEmbDict[tuple(textTrainArr[sentence][word])][0].decode('UTF-8'))
      if(flag == True):
        break
  sent2 = ' '.join(arr_words2)
  arr_words_no_comma2.append(sent2)
  arr2.append(arr_words_no_comma2)

print("Done")

Streaming output truncated to the last 5000 lines.
SENTENCE NUMBER - 10291
SENTENCE NUMBER - 10292
SENTENCE NUMBER - 10293
SENTENCE NUMBER - 10294
SENTENCE NUMBER - 10295
SENTENCE NUMBER - 10296
SENTENCE NUMBER - 10297
SENTENCE NUMBER - 10298
SENTENCE NUMBER - 10299
SENTENCE NUMBER - 10300
SENTENCE NUMBER - 10301
SENTENCE NUMBER - 10302
SENTENCE NUMBER - 10303
SENTENCE NUMBER - 10304
SENTENCE NUMBER - 10305
SENTENCE NUMBER - 10306
SENTENCE NUMBER - 10307
SENTENCE NUMBER - 10308
SENTENCE NUMBER - 10309
SENTENCE NUMBER - 10310
SENTENCE NUMBER - 10311
SENTENCE NUMBER - 10312
SENTENCE NUMBER - 10313
SENTENCE NUMBER - 10314
SENTENCE NUMBER - 10315
SENTENCE NUMBER - 10316
SENTENCE NUMBER - 10317
SENTENCE NUMBER - 10318
SENTENCE NUMBER - 10319
SENTENCE NUMBER - 10320
SENTENCE NUMBER - 10321
SENTENCE NUMBER - 10322
SENTENCE NUMBER - 10323
SENTENCE NUMBER - 10324
SENTENCE NUMBER - 10325
SENTENCE NUMBER - 10326
SENTENCE NUMBER - 10327
SENTENCE NUMBER - 10328
SENTENCE NUMBER - 10329
SENTENCE NUMB

In [ ]:
with open("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Train.csv", 'w') as csvfp: 
    csvwriter = csv.writer(csvfp)  
    csvwriter.writerow(columnName)
    csvwriter.writerows(arr2)

In [ ]:
dfTrainPart2 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Train.csv")
display(dfTrainPart2.head())

,Sentence
0,the person i am today and sp given me the oppo...
1,sp from there i went to the university of miss...
2,how companies are realizing that marketing is ...
3,how companies are realizing that marketing is ...
4,you gain from it is much higher than i think t...


In [ ]:
dfTrainPart2['Sentence Length'] = [len(x.split()) for x in dfTrainPart2['Sentence'].tolist()]
display(dfTrainPart2.head())
dfTrainPart2.to_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Train.csv", index=False)

,Sentence,Sentence Length
0,the person i am today and sp given me the oppo...,19
1,sp from there i went to the university of miss...,20
2,how companies are realizing that marketing is ...,19
3,how companies are realizing that marketing is ...,19
4,you gain from it is much higher than i think t...,20


In [3]:
dfTrainPart1 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Train.csv")
display(dfTrainPart1.head())

,Sentence,Sentence Length
0,independence we are only bound by human rights...,19
1,sp sp sp sp we scrutinize what governments wha...,20
2,sp sp sp sp sp sp sp sp sp sp sp sp sp sp inde...,20
3,sp sp sp sp work we help the states to impleme...,20
4,us to do that effectively we need to be indepe...,20


In [4]:
print(dfTrainPart1.shape)

(7645, 2)


In [5]:
dfTrainPart2 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Train.csv")
display(dfTrainPart2.head())

,Sentence,Sentence Length
0,the person i am today and sp given me the oppo...,19
1,sp from there i went to the university of miss...,20
2,how companies are realizing that marketing is ...,19
3,how companies are realizing that marketing is ...,19
4,you gain from it is much higher than i think t...,20


In [6]:
print(dfTrainPart2.shape)

(7645, 2)


In [7]:
train_df = pd.concat([dfTrainPart1, dfTrainPart2], axis=0, ignore_index=True)
display(train_df)
train_df.to_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentence-Train-Final.csv", index=False)

,Sentence,Sentence Length
0,independence we are only bound by human rights...,19
1,sp sp sp sp we scrutinize what governments wha...,20
2,sp sp sp sp sp sp sp sp sp sp sp sp sp sp inde...,20
3,sp sp sp sp work we help the states to impleme...,20
4,us to do that effectively we need to be indepe...,20
...,...,...
15285,doubt if you can show whether or not this part...,20
15286,are not and they show that a particular condit...,19
15287,sp sp it sp that the defendant the chemical co...,19
15288,the burden of proof is on the person bringing ...,20


In [ ]:
dataMosei_valid_text = h5py.File("/content/drive/MyDrive/Dissertation/dataset/data/text_valid_emb.h5", "r")

In [ ]:
text_validKeys = []

for key in dataMosei_valid_text.keys():
    text_validKeys.append(key)

print(text_validKeys)

['d1']


In [ ]:
print("-----------Create Valid array-----------")

textValidArr = np.array(dataMosei_valid_text.get("d1"))
print(textValidArr.shape)
print(textValidArr[45].shape)
print(textValidArr[67][17].shape)

-----------Create Valid array-----------
(2291, 20, 300)
(20, 300)
(300,)


In [ ]:
len(textValidArr)

2291

In [ ]:
arr3 = []
for sentence in range(1146):
  print("SENTENCE NUMBER - {}".format(sentence))
  arr_words3 = []
  arr_words_no_comma3 = []
  for word in range(20):
    for j in range(len(list(rawWordEmbDict.keys()))):
      flag = False
      if(tuple(textValidArr[sentence][word])) in rawWordEmbDict:
        flag = True
        arr_words3.append(rawWordEmbDict[tuple(textValidArr[sentence][word])][0].decode('UTF-8'))
      if(flag == True):
        break
  sent3 = ' '.join(arr_words3)
  arr_words_no_comma3.append(sent3)
  arr3.append(arr_words_no_comma3)

print("Done")

SENTENCE NUMBER - 0
SENTENCE NUMBER - 1
SENTENCE NUMBER - 2
SENTENCE NUMBER - 3
SENTENCE NUMBER - 4
SENTENCE NUMBER - 5
SENTENCE NUMBER - 6
SENTENCE NUMBER - 7
SENTENCE NUMBER - 8
SENTENCE NUMBER - 9
SENTENCE NUMBER - 10
SENTENCE NUMBER - 11
SENTENCE NUMBER - 12
SENTENCE NUMBER - 13
SENTENCE NUMBER - 14
SENTENCE NUMBER - 15
SENTENCE NUMBER - 16
SENTENCE NUMBER - 17
SENTENCE NUMBER - 18
SENTENCE NUMBER - 19
SENTENCE NUMBER - 20
SENTENCE NUMBER - 21
SENTENCE NUMBER - 22
SENTENCE NUMBER - 23
SENTENCE NUMBER - 24
SENTENCE NUMBER - 25
SENTENCE NUMBER - 26
SENTENCE NUMBER - 27
SENTENCE NUMBER - 28
SENTENCE NUMBER - 29
SENTENCE NUMBER - 30
SENTENCE NUMBER - 31
SENTENCE NUMBER - 32
SENTENCE NUMBER - 33
SENTENCE NUMBER - 34
SENTENCE NUMBER - 35
SENTENCE NUMBER - 36
SENTENCE NUMBER - 37
SENTENCE NUMBER - 38
SENTENCE NUMBER - 39
SENTENCE NUMBER - 40
SENTENCE NUMBER - 41
SENTENCE NUMBER - 42
SENTENCE NUMBER - 43
SENTENCE NUMBER - 44
SENTENCE NUMBER - 45
SENTENCE NUMBER - 46
SENTENCE NUMBER - 47
SE

In [ ]:
with open("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Valid.csv", 'w') as csvfp: 
    csvwriter = csv.writer(csvfp)  
    csvwriter.writerow(columnName)
    csvwriter.writerows(arr3)

In [ ]:
dfValidPart1 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Valid.csv")
display(dfValidPart1.head())

,Sentence
0,sp sp sp sp sp albums now you might wonder why...
1,sp sp sp sp sp sp sp sp sp sp this is another ...
2,albums like this that i always misplace or tha...
3,sp sp sp sp sp sp sp sp sp sp sp sp sp sp how ...
4,sp sp sp out there that will give you an oppor...


In [ ]:
dfValidPart1['Sentence Length'] = [len(x.split()) for x in dfValidPart1['Sentence'].tolist()]
display(dfValidPart1.head())
dfValidPart1.to_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Valid.csv", index=False)

,Sentence,Sentence Length
0,sp sp sp sp sp albums now you might wonder why...,20
1,sp sp sp sp sp sp sp sp sp sp this is another ...,20
2,albums like this that i always misplace or tha...,20
3,sp sp sp sp sp sp sp sp sp sp sp sp sp sp how ...,20
4,sp sp sp out there that will give you an oppor...,20


In [ ]:
arr4 = []
for sentence in range(1146, 2291):
  print("SENTENCE NUMBER - {}".format(sentence))
  arr_words4 = []
  arr_words_no_comma4 = []
  for word in range(20):
    for j in range(len(list(rawWordEmbDict.keys()))):
      flag = False
      if(tuple(textValidArr[sentence][word])) in rawWordEmbDict:
        flag = True
        arr_words4.append(rawWordEmbDict[tuple(textValidArr[sentence][word])][0].decode('UTF-8'))
      if(flag == True):
        break
  sent4 = ' '.join(arr_words4)
  arr_words_no_comma4.append(sent4)
  arr4.append(arr_words_no_comma4)

print("Done")

SENTENCE NUMBER - 1146
SENTENCE NUMBER - 1147
SENTENCE NUMBER - 1148
SENTENCE NUMBER - 1149
SENTENCE NUMBER - 1150
SENTENCE NUMBER - 1151
SENTENCE NUMBER - 1152
SENTENCE NUMBER - 1153
SENTENCE NUMBER - 1154
SENTENCE NUMBER - 1155
SENTENCE NUMBER - 1156
SENTENCE NUMBER - 1157
SENTENCE NUMBER - 1158
SENTENCE NUMBER - 1159
SENTENCE NUMBER - 1160
SENTENCE NUMBER - 1161
SENTENCE NUMBER - 1162
SENTENCE NUMBER - 1163
SENTENCE NUMBER - 1164
SENTENCE NUMBER - 1165
SENTENCE NUMBER - 1166
SENTENCE NUMBER - 1167
SENTENCE NUMBER - 1168
SENTENCE NUMBER - 1169
SENTENCE NUMBER - 1170
SENTENCE NUMBER - 1171
SENTENCE NUMBER - 1172
SENTENCE NUMBER - 1173
SENTENCE NUMBER - 1174
SENTENCE NUMBER - 1175
SENTENCE NUMBER - 1176
SENTENCE NUMBER - 1177
SENTENCE NUMBER - 1178
SENTENCE NUMBER - 1179
SENTENCE NUMBER - 1180
SENTENCE NUMBER - 1181
SENTENCE NUMBER - 1182
SENTENCE NUMBER - 1183
SENTENCE NUMBER - 1184
SENTENCE NUMBER - 1185
SENTENCE NUMBER - 1186
SENTENCE NUMBER - 1187
SENTENCE NUMBER - 1188
SENTENCE NU

In [ ]:
with open("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Valid.csv", 'w') as csvfp: 
    csvwriter = csv.writer(csvfp)  
    csvwriter.writerow(columnName)
    csvwriter.writerows(arr4)

In [ ]:
dfValidPart2 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Valid.csv")
display(dfValidPart2.head())

,Sentence
0,the national reserve cause stutter of course t...
1,sp gangsters or anything like that sp just you...
2,sp sp sp sp sp sp sp sp sp sp sp sp sp for the...
3,a sweepstakes or so the mobile is gonna be a b...
4,a sweepstakes or so the mobile is gonna be a b...


In [ ]:
dfValidPart2['Sentence Length'] = [len(x.split()) for x in dfValidPart2['Sentence'].tolist()]
display(dfValidPart2.head())
dfValidPart2.to_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Valid.csv", index=False)

,Sentence,Sentence Length
0,the national reserve cause stutter of course t...,20
1,sp gangsters or anything like that sp just you...,20
2,sp sp sp sp sp sp sp sp sp sp sp sp sp for the...,20
3,a sweepstakes or so the mobile is gonna be a b...,20
4,a sweepstakes or so the mobile is gonna be a b...,20


In [8]:
dfValidPart1 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Valid.csv")
display(dfValidPart1.head())

,Sentence,Sentence Length
0,sp sp sp sp sp albums now you might wonder why...,20
1,sp sp sp sp sp sp sp sp sp sp this is another ...,20
2,albums like this that i always misplace or tha...,20
3,sp sp sp sp sp sp sp sp sp sp sp sp sp sp how ...,20
4,sp sp sp out there that will give you an oppor...,20


In [9]:
print(dfValidPart1.shape)

(1146, 2)


In [10]:
dfValidPart2 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Valid.csv")
display(dfValidPart2.head())

,Sentence,Sentence Length
0,the national reserve cause stutter of course t...,20
1,sp gangsters or anything like that sp just you...,20
2,sp sp sp sp sp sp sp sp sp sp sp sp sp for the...,20
3,a sweepstakes or so the mobile is gonna be a b...,20
4,a sweepstakes or so the mobile is gonna be a b...,20


In [11]:
print(dfValidPart2.shape)

(1145, 2)


In [13]:
valid_df = pd.concat([dfValidPart1, dfValidPart2], axis=0, ignore_index=True)
display(valid_df)
valid_df.to_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentence-Valid-Final.csv", index=False)

print(valid_df.shape)

,Sentence,Sentence Length
0,sp sp sp sp sp albums now you might wonder why...,20
1,sp sp sp sp sp sp sp sp sp sp this is another ...,20
2,albums like this that i always misplace or tha...,20
3,sp sp sp sp sp sp sp sp sp sp sp sp sp sp how ...,20
4,sp sp sp out there that will give you an oppor...,20
...,...,...
2286,sp sp sp sp sp sp sp sp a beloved performer an...,20
2287,sp sp of quit smoking and i get drunk every ni...,20
2288,sp sp sp sp sp sp sp sp air maybe he was just ...,20
2289,sp sp sp sp sp as a i have three kids with my ...,20


(2291, 2)


In [ ]:
dataMosei_test_text = h5py.File("/content/drive/MyDrive/Dissertation/dataset/data/text_test_emb.h5", "r")

In [ ]:
text_testKeys = []

for key in dataMosei_test_text.keys():
    text_testKeys.append(key)

print(text_testKeys)

['d1']


In [ ]:
print("-----------Create test array-----------")

textTestArr = np.array(dataMosei_test_text.get("d1"))
print(textTestArr.shape)
print(textTestArr[45].shape)
print(textTestArr[67][17].shape)

-----------Create test array-----------
(4832, 20, 300)
(20, 300)
(300,)


In [ ]:
len(textTestArr)

4832

In [ ]:
arr5 = []
for sentence in range(2416):
  print("SENTENCE NUMBER - {}".format(sentence))
  arr_words5 = []
  arr_words_no_comma5 = []
  for word in range(20):
    for j in range(len(list(rawWordEmbDict.keys()))):
      flag = False
      if(tuple(textTestArr[sentence][word])) in rawWordEmbDict:
        flag = True
        arr_words5.append(rawWordEmbDict[tuple(textTestArr[sentence][word])][0].decode('UTF-8'))
      if(flag == True):
        break
  sent5 = ' '.join(arr_words5)
  arr_words_no_comma5.append(sent5)
  arr5.append(arr_words_no_comma5)

print("Done")

SENTENCE NUMBER - 0
SENTENCE NUMBER - 1
SENTENCE NUMBER - 2
SENTENCE NUMBER - 3
SENTENCE NUMBER - 4
SENTENCE NUMBER - 5
SENTENCE NUMBER - 6
SENTENCE NUMBER - 7
SENTENCE NUMBER - 8
SENTENCE NUMBER - 9
SENTENCE NUMBER - 10
SENTENCE NUMBER - 11
SENTENCE NUMBER - 12
SENTENCE NUMBER - 13
SENTENCE NUMBER - 14
SENTENCE NUMBER - 15
SENTENCE NUMBER - 16
SENTENCE NUMBER - 17
SENTENCE NUMBER - 18
SENTENCE NUMBER - 19
SENTENCE NUMBER - 20
SENTENCE NUMBER - 21
SENTENCE NUMBER - 22
SENTENCE NUMBER - 23
SENTENCE NUMBER - 24
SENTENCE NUMBER - 25
SENTENCE NUMBER - 26
SENTENCE NUMBER - 27
SENTENCE NUMBER - 28
SENTENCE NUMBER - 29
SENTENCE NUMBER - 30
SENTENCE NUMBER - 31
SENTENCE NUMBER - 32
SENTENCE NUMBER - 33
SENTENCE NUMBER - 34
SENTENCE NUMBER - 35
SENTENCE NUMBER - 36
SENTENCE NUMBER - 37
SENTENCE NUMBER - 38
SENTENCE NUMBER - 39
SENTENCE NUMBER - 40
SENTENCE NUMBER - 41
SENTENCE NUMBER - 42
SENTENCE NUMBER - 43
SENTENCE NUMBER - 44
SENTENCE NUMBER - 45
SENTENCE NUMBER - 46
SENTENCE NUMBER - 47
SE

In [ ]:
with open("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Test.csv", 'w') as csvfp: 
    csvwriter = csv.writer(csvfp)  
    csvwriter.writerow(columnName)
    csvwriter.writerows(arr5)

In [ ]:
dfTestPart1 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Test.csv")
display(dfTestPart1.head())

,Sentence
0,what doing sp not talking about just going int...
1,super connectors that help businesses via thei...
2,sp sp sp sp now you see this online with big b...
3,sp sp sp sp sp sp sp benefits the first being ...
4,sp sp sp sp sp sp sp sp sp them so how do you ...


In [ ]:
dfTestPart1['Sentence Length'] = [len(x.split()) for x in dfTestPart1['Sentence'].tolist()]
display(dfTestPart1.head())
dfTestPart1.to_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Test.csv", index=False)

,Sentence,Sentence Length
0,what doing sp not talking about just going int...,19
1,super connectors that help businesses via thei...,20
2,sp sp sp sp now you see this online with big b...,20
3,sp sp sp sp sp sp sp benefits the first being ...,19
4,sp sp sp sp sp sp sp sp sp them so how do you ...,20


In [ ]:
arr6 = []
for sentence in range(2416, 4832):
  print("SENTENCE NUMBER - {}".format(sentence))
  arr_words6 = []
  arr_words_no_comma6 = []
  for word in range(20):
    for j in range(len(list(rawWordEmbDict.keys()))):
      flag = False
      if(tuple(textTestArr[sentence][word])) in rawWordEmbDict:
        flag = True
        arr_words6.append(rawWordEmbDict[tuple(textTestArr[sentence][word])][0].decode('UTF-8'))
      if(flag == True):
        break
  sent6 = ' '.join(arr_words6)
  arr_words_no_comma6.append(sent6)
  arr6.append(arr_words_no_comma6)

print("Done")

SENTENCE NUMBER - 2416
SENTENCE NUMBER - 2417
SENTENCE NUMBER - 2418
SENTENCE NUMBER - 2419
SENTENCE NUMBER - 2420
SENTENCE NUMBER - 2421
SENTENCE NUMBER - 2422
SENTENCE NUMBER - 2423
SENTENCE NUMBER - 2424
SENTENCE NUMBER - 2425
SENTENCE NUMBER - 2426
SENTENCE NUMBER - 2427
SENTENCE NUMBER - 2428
SENTENCE NUMBER - 2429
SENTENCE NUMBER - 2430
SENTENCE NUMBER - 2431
SENTENCE NUMBER - 2432
SENTENCE NUMBER - 2433
SENTENCE NUMBER - 2434
SENTENCE NUMBER - 2435
SENTENCE NUMBER - 2436
SENTENCE NUMBER - 2437
SENTENCE NUMBER - 2438
SENTENCE NUMBER - 2439
SENTENCE NUMBER - 2440
SENTENCE NUMBER - 2441
SENTENCE NUMBER - 2442
SENTENCE NUMBER - 2443
SENTENCE NUMBER - 2444
SENTENCE NUMBER - 2445
SENTENCE NUMBER - 2446
SENTENCE NUMBER - 2447
SENTENCE NUMBER - 2448
SENTENCE NUMBER - 2449
SENTENCE NUMBER - 2450
SENTENCE NUMBER - 2451
SENTENCE NUMBER - 2452
SENTENCE NUMBER - 2453
SENTENCE NUMBER - 2454
SENTENCE NUMBER - 2455
SENTENCE NUMBER - 2456
SENTENCE NUMBER - 2457
SENTENCE NUMBER - 2458
SENTENCE NU

In [ ]:
with open("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Test.csv", 'w') as csvfp: 
    csvwriter = csv.writer(csvfp)  
    csvwriter.writerow(columnName)
    csvwriter.writerows(arr6)

In [ ]:
dfTestPart2 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Test.csv")
display(dfTestPart2.head())

,Sentence
0,sp sp sp sp sp sp they worry about the state d...
1,of the school districts in pennsylvania who ar...
2,sp sp sp sp sp sp sp sp sp i will work toward ...
3,sp sp as the only candidate with a military an...
4,sp i will furthermore address the proliferatio...


In [ ]:
dfTestPart2['Sentence Length'] = [len(x.split()) for x in dfTestPart2['Sentence'].tolist()]
display(dfTestPart2.head())
dfTestPart2.to_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Test.csv", index=False)

,Sentence,Sentence Length
0,sp sp sp sp sp sp they worry about the state d...,19
1,of the school districts in pennsylvania who ar...,19
2,sp sp sp sp sp sp sp sp sp i will work toward ...,20
3,sp sp as the only candidate with a military an...,20
4,sp i will furthermore address the proliferatio...,20


In [14]:
dfTestPart1 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart1-Test.csv")
display(dfTestPart1.head())

,Sentence,Sentence Length
0,what doing sp not talking about just going int...,19
1,super connectors that help businesses via thei...,20
2,sp sp sp sp now you see this online with big b...,20
3,sp sp sp sp sp sp sp benefits the first being ...,19
4,sp sp sp sp sp sp sp sp sp them so how do you ...,20


In [15]:
print(dfTestPart1.shape)

(2416, 2)


In [16]:
dfTestPart2 = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentencePart2-Test.csv")
display(dfTestPart2.head())

,Sentence,Sentence Length
0,sp sp sp sp sp sp they worry about the state d...,19
1,of the school districts in pennsylvania who ar...,19
2,sp sp sp sp sp sp sp sp sp i will work toward ...,20
3,sp sp as the only candidate with a military an...,20
4,sp i will furthermore address the proliferatio...,20


In [17]:
print(dfTestPart2.shape)

(2416, 2)


In [18]:
test_df = pd.concat([dfTestPart1, dfTestPart2], axis=0, ignore_index=True)
display(test_df)
test_df.to_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentence-Test-Final.csv", index=False)

print(test_df.shape)

,Sentence,Sentence Length
0,what doing sp not talking about just going int...,19
1,super connectors that help businesses via thei...,20
2,sp sp sp sp now you see this online with big b...,20
3,sp sp sp sp sp sp sp benefits the first being ...,19
4,sp sp sp sp sp sp sp sp sp them so how do you ...,20
...,...,...
4827,sp sp sp sp sp sp sp sp sp sp call or email me...,20
4828,in addition investment property has a home lik...,20
4829,you will give yourself a sure footing and be o...,20
4830,sp sp sp sp sp sp sp sp sp sp sp sp sp for the...,20


(4832, 2)
